Start here: https://en.wikipedia.org/wiki/List_of_men%27s_national_association_football_teams

In [36]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

import time

from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [3]:
team_list_page = requests.get("https://en.wikipedia.org/wiki/List_of_men's_national_association_football_teams")
team_list_page.raise_for_status()

### Extract links to all football team wiki pages

In [4]:
teams_soup = BeautifulSoup(team_list_page.text, "html.parser")
team_anchors = teams_soup.select("span.flagicon+a")

team_links = []

for tag in team_anchors:
    team_links.append(tag["href"])

team_links = list(set(team_links))
team_links

['/wiki/Dominica_national_football_team',
 '/wiki/Ecuador_national_football_team',
 '/wiki/Kuwait_national_football_team',
 '/wiki/Republic_of_Ireland_national_football_team',
 '/wiki/Pakistan_national_football_team',
 '/wiki/Spain_national_football_team',
 '/wiki/Slovakia_national_football_team',
 '/wiki/Slovenia_national_football_team',
 '/wiki/Luxembourg_national_football_team',
 '/wiki/Tonga_national_football_team',
 '/wiki/Switzerland_national_football_team',
 '/wiki/Palestine_national_football_team',
 '/wiki/France_national_football_team',
 '/wiki/Ukraine_national_football_team',
 '/wiki/Cura%C3%A7ao_national_football_team',
 '/wiki/Eswatini_national_football_team',
 '/wiki/Somalia_national_football_team',
 '/wiki/Estonia_national_football_team',
 '/wiki/Cura%C3%A7ao_national_football_team_(1921%E2%80%931958)',
 '/wiki/Comoros_national_football_team',
 '/wiki/Anguilla_national_football_team',
 '/wiki/Finland_national_football_team',
 '/wiki/Colombia_national_football_team',
 '/wi

### Extract most capped player for each country

Define a function to extract most capped player from a country's page and call it on all team pages

In [5]:
BASE_URL = "https://en.wikipedia.org"

def get_most_capped_player(country_link):
    team_full_link = f"{BASE_URL}{country_link}"
    #print(team_full_link)

    team_page = requests.get(team_full_link)
    team_page.raise_for_status()
    
    team_soup = BeautifulSoup(team_page.text, "html.parser")

    info_rows = team_soup.select("tr:has(th.infobox-label)")

    for info_row in info_rows:
        row_label = info_row.select("th.infobox-label")[0].text
        if row_label == "Most caps":
            player_tag = info_row.select("td.infobox-data a")[0]
            print(f"Most capped player: {player_tag.get_text()}")
            player_page = player_tag["href"]
            return player_page
    
    raise ValueError("Most capped player not found")

In [6]:
get_most_capped_player(team_links[0])

Most capped player: Glenson Prince


'/wiki/Glenson_Prince'

Define a function to extract the clubs the most capped player has played for

In [50]:
def get_club_list(player_link):
    player_full_link = f"{BASE_URL}{player_link}"
    #print(player_full_link)
    player_page = requests.get(player_full_link)
    player_page.raise_for_status()
    
    player_soup = BeautifulSoup(player_page.text, "html.parser")
    
    # vcard is the name of the stats table on the wiki page
    vcard = player_soup.select("table.infobox.vcard")[0]

    vcard_df = pd.read_html(str(vcard))[0]

    clubs_start_idx = (
        vcard_df
        .dropna(subset=[vcard_df.columns[0]])
        [vcard_df[vcard_df.columns[0]].dropna().str.contains("Senior career")]
        .index[0]
    )
    
    clubs_end_idx = (
        vcard_df
        .dropna(subset=[vcard_df.columns[0]])
        [vcard_df[vcard_df.columns[0]].dropna().str.contains("International career")]
        .index[0]
    )
    
    clubs = vcard_df.loc[clubs_start_idx+2:clubs_end_idx-1,:]
    clubs.columns = ["Date", "Team", "Appearances", "Goals"]
    return clubs

In [51]:
get_club_list(get_most_capped_player(team_links[0]))

Most capped player: Glenson Prince


,Date,Team,Appearances,Goals
9,–2014,Northern Bombers,NaN,NaN
10,2014,San Juan Jabloteh,NaN,NaN
11,2015–2016,Northern Bombers,NaN,NaN
12,2016–2017,Dublanc FC,NaN,NaN
13,2017–2021,Phare Petit-Canal[3],NaN,NaN
14,2021–,Dublanc FC,NaN,NaN


In [52]:
errors = []

for team_link in team_links:
    try:
        team_name = team_link.split("/")[-1].replace("_national_football_team", "").replace("_", "")
        print(f"{team_name}")
        most_capped_player_link = get_most_capped_player(team_link)
        
        player_clubs = get_club_list(most_capped_player_link)
        display(player_clubs)
    
        print("\n\n###############################################\n\n")
        time.sleep(0.1)
    except:
        print(f"Could not extract info for {team_link}")
        errors.append(team_link)

Dominica
Most capped player: Glenson Prince


,Date,Team,Appearances,Goals
9,–2014,Northern Bombers,NaN,NaN
10,2014,San Juan Jabloteh,NaN,NaN
11,2015–2016,Northern Bombers,NaN,NaN
12,2016–2017,Dublanc FC,NaN,NaN
13,2017–2021,Phare Petit-Canal[3],NaN,NaN
14,2021–,Dublanc FC,NaN,NaN




###############################################


Ecuador
Most capped player: Iván Hurtado


,Date,Team,Appearances,Goals
9,1992–1996,Emelec,102,(10)
10,1996–1999,Celaya,89,(2)
11,1999–2001,Tigres,79,(0)
12,2001,Emelec,14,(1)
13,2001–2002,Querétaro,18,(0)
14,2002–2004,Barcelona SC,57,(2)
15,2004,Murcia,15,(0)
16,2004–2005,Pachuca,26,(0)
17,2005–2006,Al Arabi,13,(3)
18,2006,Al-Ahly,0,(0)




###############################################


Kuwait
Most capped player: Bader Al-Mutawa


...



###############################################


Niuenationalsoccerteam
Most capped player: Bradley Punu
Could not extract info for /wiki/Niue_national_soccer_team
Somaliland
Could not extract info for /wiki/Somaliland_national_football_team
Greece
Most capped player: Giorgos Karagounis


,Date,Team,Appearances,Goals
11,1996–2003,Panathinaikos,118,(25)
12,1996–1998,→ Apollon Athens (loan),55,(9)
13,2003–2005,Inter Milan,21,(0)
14,2005–2007,Benfica,45,(3)
15,2007–2012,Panathinaikos,133,(16)
16,2012–2014,Fulham,39,(1)
17,Total,NaN,411,(54)




###############################################


Burundi
Most capped player: Karim Nizigiyimana


,Date,Team,Appearances,Goals
11,2006,Vital'O,20,(0)
12,2008–2009,A.P.R.,11,(0)
13,2009–2010,Kiyovu Sports,9,(0)
14,2010,Vita Club,0,(0)
15,2010–2011,Vital'O,NaN,NaN
16,2011–2015,Rayon Sports,NaN,NaN
17,2015–2018,Gor Mahia,NaN,NaN
18,2019,Vipers,NaN,NaN
19,2019–,Wazito,NaN,NaN




###############################################


